# Analyze population data from www.data.gouv.fr and/or INSEE

**Note:** This is a Jupyter notebook which is also available as its executable export as a Python 3 script (therefore with automatically generated comments).

# Libraries

In [1]:
# Sys import
import sys, os, re
# Common imports
import math
import numpy             as NP
import numpy.random      as RAND
import scipy.stats       as STATS
from scipy import sparse
from scipy import linalg

# Better formatting functions
from IPython.display import display, HTML
from IPython import get_ipython

import matplotlib        as MPL
import matplotlib.pyplot as PLT
import seaborn as SNS
SNS.set(font_scale=1)

# Python programming
from itertools import cycle
from time import time
import datetime

# Using pandas
import pandas as PAN
import xlrd

In [2]:
import warnings
warnings.filterwarnings('ignore')
print("For now, reduce python warnings, I will look into this later")

For now, reduce python warnings, I will look into this later


### Import my own modules
The next cell attempts to give user some information if things improperly setup.
Intended to work both in Jupyter and when executing the Python file directly.

In [3]:
if not get_ipython() is None and os.path.abspath("../source/") not in sys.path:
    sys.path.append(os.path.abspath("../source/"))
try:
    from lib.utilities     import *
    from lib.figureHelpers import *
    from lib.DataGouvFr    import *
    from lib.pandaUtils    import *
except Exception as err:
    print("Could not find library 'lib' with contents 'DataGouvFr' ")
    if get_ipython() is None:
        print("Check the PYTHONPATH environment variable which should point to 'source' wich contains 'lib'")
    else:
        print("You are supposed to be running in JupySessions, and '../source/lib' should exist")
    raise err

Loaded lib
For now, reduce python warnings, I will look into this later


## Check environment

It is expected that:
- your working directory is named `JupySessions`, 
- that it has subdirectories 
   - `images/*` where generated images may be stored to avoid overcrowding. 
- At the same level as your working dir there should be directories 
   - `../data` for storing input data and 
   - `../source` for python scripts.
   
My package library is in `../source/lib`, and users running under Python (not in Jupyter) should
set their PYTHONPATH to include "../source" ( *or whatever appropriate* ).

In [4]:
checkSetup(chap="Chap02")
ImgMgr = ImageMgr(chapdir="Chap02")

# Load Data

## Functions

## Load CSV and XLSX data from remote 
The `dataFileVMgr` will manage a cache of data files in `../dataPop`.

We check what is in the cache/data directory; for each file, we identify the latest version, 
and list this below to make sure. The file name will usually contain a time stamp; this has to do with 
the version management/identification technique used when downloading from www.data.gouv.fr or INSEE

For the files used in this notebook, the latest version is used/loaded irrespective of the
timestamp used in the notebook.

Consulted:
  - https://www.insee.fr/fr/information/2410988
  - https://www.insee.fr/fr/information/2008354
  - https://www.insee.fr/fr/statistiques/4265429?sommaire=4265511#consulter  
  - https://www.insee.fr/fr/statistiques/4265429?sommaire=4265511#documentation
  - https://api.insee.fr/catalogue/
  - https://api.insee.fr/catalogue/site/themes/wso2/subthemes/insee/pages/help.jag
  
Downloaded:
  
  - https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-gca-1975-2020.xls
  - https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-aq-1975-2020.xls
  - https://www.insee.fr/fr/statistiques/fichier/1893198/evolution-population-dep-2007-2020.xls
  - https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-nreg-sexe-gca-1975-2020.xls
  - https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-nreg-sexe-aq-1975-2020.xls
  - https://www.insee.fr/fr/statistiques/fichier/1893198/evolution-population-nreg-2007-2020.xls

In [5]:
dataFileVMgr = manageAndCacheFilesDFrame("../dataPop")

In [6]:
print("Most recent versions of files in data directory:")
for f in dataFileVMgr.listMostRecent() :
    print(f"\t{f}")

Most recent versions of files in data directory:
	export-dataset-20200314-064905.csv
	export-organization-20200314-065744.csv
	tags-2020-04-20-09-22.csv


In [7]:
last = lambda x: dataFileVMgr.getRecentVersion(x,default=True)

This ensures we load the most recent version, so that it is not required to update the list 
below. The timestamps shown in the following sequence will be update by the call to `getRecentVersion`.

In [8]:
exportDSCsv    = last("export-dataset-20200314-064905.csv")
exportORGCsv    = last("export-organization-20200314-065744.csv")
tagsCsv        = last("tags-2020-04-20-09-22.csv")


In [9]:
S1 = set (dataFileVMgr.listMostRecent())
S2 =set((exportDSCsv ,tagsCsv,  exportORGCsv ))
missing = S1. difference(S2)
if len(missing) > 0:
    print (f"Missing comparing with most recent files in ../data:")
    for f in missing:
        print(f"\t{f}")

In [10]:
metaCodeTranchesAgeCSV ="../dataPop/code-tranches-dage.csv" 

estimPopDepSxXLS      ="../dataPop/estim-pop-dep-sexe-aq-1975-2020.xls"
estimPopDepSxGXLS     ="../dataPop/estim-pop-dep-sexe-gca-1975-2020.xls"
estimPopRegSxXLS      ="../dataPop/estim-pop-nreg-sexe-aq-1975-2020.xls"
estimPopRegSxGXLS     ="../dataPop/estim-pop-nreg-sexe-gca-1975-2020.xls"
evolPopDepXLS         ="../dataPop/evolution-population-dep-2007-2020.xls"
evolPopRegXLS         ="../dataPop/evolution-population-nreg-2007-2020.xls"

inseeDepXLS           ="../dataPop/InseeDep.xls"

Now load the stuff

In [11]:
ad  = lambda x: "../dataPop/"+x
data_exportDS = read_csvPandas(ad(exportDSCsv), error_bad_lines=False,sep=";" )
data_tags     = read_csvPandas(ad(tagsCsv), error_bad_lines=False,sep=";")
data_exportORG= read_csvPandas(ad(exportORGCsv), error_bad_lines=False,sep=";")

In [12]:
metaCodeTranchesAge = read_csvPandas(metaCodeTranchesAgeCSV, error_bad_lines=False,sep=";")
estimPopDepSx       = read_xlsxPandas(estimPopDepSxXLS, error_bad_lines=False,sep=",")
estimPopDepSxG      = read_xlsxPandas(estimPopDepSxGXLS, error_bad_lines=False,sep=",")
estimPopRegSxX      = read_xlsxPandas(estimPopRegSxXLS , error_bad_lines=False,sep=",")
estimPopRegSxG      = read_xlsxPandas(estimPopRegSxGXLS, error_bad_lines=False,sep=",")
evolPopDep          = read_xlsxPandas(evolPopDepXLS , error_bad_lines=False,sep=",")       
evolPopReg          = read_xlsxPandas(evolPopRegXLS , error_bad_lines=False,sep=",")      

inseeDep            = read_xlsxPandas(inseeDepXLS, error_bad_lines=False,sep=",", sheet_name=1, header=7)
inseeReg            = read_xlsxPandas(inseeDepXLS, error_bad_lines=False,sep=",", sheet_name=0, header=7)

Show the shape of the loaded data:

In [13]:
def showBasics(data,dataName):
    print(f"{dataName:24}\thas shape {data.shape}")

dataListDescr = ( (data_exportDS, "data_exportDS"),
                (data_tags, "data_tags"),
                (data_exportORG,"data_exportORG"),
                (metaCodeTranchesAge,"metaCodeTranchesAge"),
                (estimPopDepSx ,"estimPopDepSx "),
                (estimPopDepSxG,"estimPopDepSxG"),
                (estimPopRegSxX,"estimPopRegSxX"),
                (estimPopRegSxG,"estimPopRegSxG"),
                (evolPopDep,"evolPopDep"),
                (evolPopReg,"evolPopReg"),
                (inseeDep,"inseeDep"),
                (inseeReg,"inseeReg")
                )
    
for (dat,name) in dataListDescr:
    showBasics(dat,name)


data_exportDS           	has shape (41106, 23)
data_tags               	has shape (26556, 4)
data_exportORG          	has shape (2528, 17)
metaCodeTranchesAge     	has shape (6, 2)
estimPopDepSx           	has shape (10, 1)
estimPopDepSxG          	has shape (8, 1)
estimPopRegSxX          	has shape (10, 1)
estimPopRegSxG          	has shape (8, 1)
evolPopDep              	has shape (109, 5)
evolPopReg              	has shape (26, 5)
inseeDep                	has shape (100, 9)
inseeReg                	has shape (17, 7)


In [14]:
for (dat,name) in dataListDescr:
    if name[0:5]=="meta_": continue
    print(f"\nDescription of data in '{name}'\n")
    display(dat.describe().transpose())
    


Description of data in 'data_exportDS'



,count,mean,std,min,25%,50%,75%,max
metric.discussions,41106.0,0.107600,2.037545,0.0,0.0,0.0,0.0,256.0
metric.followers,41106.0,0.111808,1.325745,0.0,0.0,0.0,0.0,124.0
metric.reuses,41106.0,0.078553,0.813452,0.0,0.0,0.0,0.0,62.0
metric.issues,41106.0,0.008125,0.129683,0.0,0.0,0.0,0.0,9.0
metric.views,41106.0,112.861431,2927.409140,0.0,0.0,0.0,2.0,263144.0



Description of data in 'data_tags'



,count,mean,std,min,25%,50%,75%,max
datasets,26556.0,9.825802,162.148386,0.0,1.0,1.0,3.0,17136.0
reuses,26556.0,0.565748,2.957556,0.0,0.0,0.0,0.0,145.0
total,26556.0,10.391550,162.400998,1.0,1.0,1.0,4.0,17137.0



Description of data in 'data_exportORG'



...

,count,unique,top,freq
Évolution de la population par région de 2007 à 2012,23,23,Hauts-de-France,1
Unnamed: 1,22,22,5973098,1
Unnamed: 2,23,14,0.005,4
Unnamed: 3,22,12,0.002,4
Unnamed: 4,22,17,0.001,3



Description of data in 'inseeDep'



,count,mean,std,min,25%,50%,75%,max
Code région,100.0,53.120000,28.086188,1.0,27.00,52.0,76.00,94.0
Nombre d'arrondissements,100.0,3.320000,0.983500,1.0,3.00,3.0,4.00,7.0
Nombre de cantons,97.0,21.041237,5.656702,9.0,17.00,21.0,23.00,41.0
Nombre de communes,100.0,349.530000,188.988293,1.0,239.75,324.5,475.25,890.0
Population municipale,100.0,665243.390000,508347.223485,76601.0,299035.00,534154.5,831792.00,2604361.0
Population totale,100.0,677610.920000,513500.212754,80240.0,306481.25,546162.5,847226.75,2635255.0



Description of data in 'inseeReg'



,count,mean,std,min,25%,50%,75%,max
Code région,17.0,4.135294e+01,3.315106e+01,1.0,11.0,32.0,75.0,94.0
Nombre d'arrondissements,17.0,1.952941e+01,1.344952e+01,2.0,5.0,18.0,26.0,41.0
Nombre de cantons,15.0,1.360000e+02,7.764204e+01,21.0,102.0,131.0,177.5,258.0
Nombre de communes,17.0,2.056059e+03,1.817808e+03,22.0,360.0,1268.0,3789.0,5121.0
Population municipale,17.0,3.913196e+06,3.209913e+06,268700.0,853659.0,3330478.0,5845102.0,12174880.0
Population totale,17.0,3.985947e+06,3.254152e+06,271124.0,863063.0,3407564.0,5968795.0,12291279.0


## Explore the INSEE export-dataset-\*.csv

### Use utilities to find rows corresponding to regexp

In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [16]:
data_exportDS.loc[:2].values

array([['5e6c221906e3e72d007168cd',
        'Prestation de Compensation du Handicap en Maine-et-Loire',
        'prestation-de-compensation-du-handicap-en-maine-et-loire',
        'http://www.data.gouv.fr/fr/datasets/prestation-de-compensation-du-handicap-en-maine-et-loire/',
        'Département de Maine-et-Loire', '5e32cef58b4c411011bd3bab',
        "Nombre de personnes en situation de handicap qui bénéficient soit d'une Prestation de Compensation du Handicap (PCH), soit d'une Allocation Compensatrice (ACTP) depuis 2016.\n\nLe secret statistique est appliqué : toutes les informations contenant moins de 5 individus ne sont pas renseignées (cellule Bénéficiaires vide).",
        'unknown', 'Licence Ouverte / Open Licence version 2.0', nan,
        nan, nan, nan, False, False, '2020-03-14T01:15:21.814000',
        '2020-03-14T01:15:21.814000',
        'aide-sociale,handicap,pch,personne-handicapee,prestation,sante-social',
        0, 0, 0, 0, 0],
       ['5e6c221906e3e72d007168cc',
    

This selects rows according to a *regular expression* (compliant with the python `re` module)

In [17]:
df1 =  DfMatcher(data_exportDS)
df1.showMatch('description','stat.*population.*')
df1.prMatch('description','stat.*population.*')

(97, 23)


,id,title,slug,url,organization,organization_id,description,frequency,license,temporal_coverage.start,...,private,featured,created_at,last_modified,tags,metric.discussions,metric.followers,metric.reuses,metric.issues,metric.views
1222,5dfc47be06e3e75fd3fac9cf,Indicateur sur la part des prestations dans le...,indicateur-sur-la-part-des-prestations-dans-le...,http://www.data.gouv.fr/fr/datasets/indicateur...,Caisse Nationale des Allocations familiales,5595066cc751df4582a453ba,__A partir de 2018 les données du mois de droi...,NaN,Licence Ouverte / Open Licence,NaN,...,False,False,2019-12-19T00:00:00,2020-02-25T00:00:00,epci,0,0,0,0,0
2786,5d3bcc5606e3e77c98e75024,Foyers allocataires percevant la prestation d'...,foyers-allocataires-percevant-la-prestation-d-...,http://www.data.gouv.fr/fr/datasets/foyers-all...,Caisse Nationale des Allocations familiales,5595066cc751df4582a453ba,__A partir de 2018 les données du mois de droi...,NaN,Licence Ouverte / Open Licence,NaN,...,False,False,2019-07-26T00:00:00,2020-02-25T00:00:00,"clca,cmg,epci,paje",0,0,0,0,2
5141,5cddfddd06e3e7152e835fa5,ALLOCATIONS - Part_prestations_ressources_foye...,allocations-part-prestations-ressources-foyers...,http://www.data.gouv.fr/fr/datasets/allocation...,Commune de Brocas,534fff65a3a7292c64a77d76,Il s’agit d’estimer la part des prestations en...,unknown,Licence Ouverte / Open Licence version 2.0,NaN,...,False,False,2019-05-17T02:18:37.320000,2019-07-12T02:02:14.596000,"administration,citizenship,government,prestati...",0,0,0,0,0
5862,5caaf1c106e3e71842d11073,Population par canton (recensement 2013),population-par-canton-recensement-2013-1,http://www.data.gouv.fr/fr/datasets/population...,Région Occitanie / Pyrénées Méditerranée,5a153f5688ee3829e2fbf39f,Population municipale et totale 2013. La popul...,unknown,Open Data Commons Open Database License (ODbL),NaN,...,False,False,2019-04-08T09:01:20.923000,2019-07-12T02:16:59.773000,territoire,0,0,0,0,0
5948,5c9c98278b4c417c4a677dee,Les personnels des SDIS et de la sécurité civile,les-personnels-des-sdis-et-de-la-securite-civile,http://www.data.gouv.fr/fr/datasets/les-person...,Cour des comptes,53698dada3a729239d20331d,La Cour et 13 chambres régionales des comptes ...,punctual,Open Data Commons Open Database License (ODbL),NaN,...,False,False,2019-03-28T10:47:19.597000,2019-03-28T10:48:00.310000,"sdis,securite-civile",0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39888,5369996ea3a729239d205250,Localisation des services résidentiels : analy...,localisation-des-services-residentiels-analyse...,http://www.data.gouv.fr/fr/datasets/localisati...,Premier ministre,534fffa5a3a7292c64a7809e,La géographie des services constitue un domain...,unknown,Licence Ouverte / Open Licence,NaN,...,False,False,2013-07-08T18:21:28.481000,2015-11-27T07:34:25.638000,NaN,0,0,0,0,1
40063,5369a0cea3a729239d206454,Statistiques relatives aux volontaires de soli...,statistiques-relatives-aux-volontaires-de-soli...,http://www.data.gouv.fr/fr/datasets/statistiqu...,Ministère de l'Europe et des Affaires Etrangèr...,534fff94a3a7292c64a77fb5,Ces statistiques 2010 sont réalisées à partir ...,annual,Licence Ouverte / Open Licence,2010-01-01,...,False,False,2013-07-08T18:13:51.666000,2014-11-05T01:56:10.622000,"association,solidarite,volontaires",0,0,0,0,2
40435,536997f4a3a729239d204e1a,Les bénéficiaires de l’allocation compensatric...,les-beneficiaires-de-l-allocation-compensatric...,http://www.data.gouv.fr/fr/datasets/les-benefi...,Ministère du travail,534fff96a3a7292c64a77ff4,Enquête menée en 2009-2010 auprès d’un échanti...,unknown,Licence Ouverte / Open Licence,2009-10-01,...,False,False,2013-07-08T15:53:08.669000,2016-02-03T06:33:00.970000,NaN,0,0,0,0,2
40454,536997b8a3a729239d204d7e,La profession d’infirmière : situation démogra...,la-profession-d-infirmiere-situation-demograph...,http://www.data.gouv.fr/fr/datasets/la-profess...,Ministère des Solidarités et de la Santé,534fff94a3a7292c64a77f

(97, 23)
__A partir de 2018 les données du mois de droit de décembre sont observées avec un recul de 6 mois (données définitives) au lieu de 2 mois (données se
__A partir de 2018 les données du mois de droit de décembre sont observées avec un recul de 6 mois (données définitives) au lieu de 2 mois (données se
Il s’agit d’estimer la part des prestations entrant dans les ressources des foyers allocataires. Pour ces données portant sur les revenus des allocata
Population municipale et totale 2013. La population municipale est exploitée usuellement en statistique (chaque personne est comptée une fois et une s
La Cour et 13 chambres régionales des comptes ont mené auprès de la direction générale de la sécurité civile et de la gestion des crises (DGSCGC) et d
Ce jeu de données contient une estimation statistique de la population par bâtiment ou groupe de bâtiment dans le département du Doubs. Cette estimati
Statistiques sur la population de la ville d'Agen.
Caractéristiques des logements et 

### Now look at population statistics
#### Landed at the wrong place
I like the atlantic seaside, but this was not the target!

In [18]:
interesting = df1.rowMatch('slug','recensement.*population-population$') 
interesting.loc[:, ['url', 'title']].values
interesting.values

array([['5bb6b4189ce2e77ee02456a9',
        'Recensement de la population – Population',
        'recensement-de-la-population-population',
        'http://www.data.gouv.fr/fr/datasets/recensement-de-la-population-population/',
        'La Rochelle', '5821930b88ee386862c65bb3',
        "Thème 'population' du RP 2008", nan, 'License Not Specified',
        nan, nan, nan, nan, False, False, '2018-10-05T02:45:12.188000',
        '2019-07-12T02:06:19.664000',
        ',denombrement,habitant,insee,population,recensement', 0, 0, 0,
        0, 1]], dtype=object)

## Explore population from INSEE files
See above for the files and how they may be found. 

In [19]:
evolPopDep          

,Évolution de la population par département de 2007 à 2012,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,Département,Estimations de population au 1er janvier 2012,Variation relative annuelle 2007-2012 (en %),NaN,NaN
2,NaN,NaN,Totale,Due au solde naturel,Due au solde apparent des entrées et des sorties
3,Ain,612191,0.013,0.005,0.008
4,Aisne,540888,0.001,0.003,-0.002
...,...,...,...,...,...
104,La Réunion,833944,0.01,0.013,-0.003
105,DOM,1865270,0.007,0.011,-0.004
106,France métropolitaine et DOM,65241241,0.005,0.004,0.001
107,NaN,NaN,NaN,NaN,NaN


Comments to be exploited later:
- evolPopReg           
- estimPopbeDepSx      
- estimPopDepSxG     
- estimPopRegSxX     
- estimPopRegSxG      

In [20]:
metaCodeTranchesAge  

,Code tranches d'age,Unnamed: 1
0,0,tous âges
1,A,moins de 15 ans
2,B,15-44 ans
3,C,45-64 ans
4,D,65-74 ans
5,E,75 et plus


In [21]:
inseeReg

,Code région,Nom de la région,Nombre d'arrondissements,Nombre de cantons,Nombre de communes,Population municipale,Population totale
0,84,Auvergne-Rhône-Alpes,39,241.0,4030,7948287,8120668
1,27,Bourgogne-Franche-Comté,24,152.0,3704,2811423,2885864
2,53,Bretagne,15,102.0,1208,3318904,3407564
3,24,Centre-Val de Loire,20,102.0,1758,2576252,2634852
4,94,Corse,5,26.0,360,334938,340233
5,44,Grand Est,38,200.0,5121,5549586,5657093
6,1,Guadeloupe,2,21.0,32,390253,396153
7,3,Guyane,2,NaN,22,268700,271124
8,32,Hauts-de-France,26,145.0,3789,6003815,6096177
9,11,Île-de-France,25,155.0,1268,12174880,12291279


In [22]:
inseeDep

,Code région,Nom de la région,Code département,Nom du département,Nombre d'arrondissements,Nombre de cantons,Nombre de communes,Population municipale,Population totale
0,84,Auvergne-Rhône-Alpes,01,Ain,4,23.0,393,643350,659180
1,32,Hauts-de-France,02,Aisne,5,21.0,800,534490,546527
2,84,Auvergne-Rhône-Alpes,03,Allier,3,19.0,317,337988,347035
3,93,Provence-Alpes-Côte d'Azur,04,Alpes-de-Haute-Provence,4,15.0,198,163915,168381
4,93,Provence-Alpes-Côte d'Azur,05,Hautes-Alpes,2,15.0,162,141284,145883
...,...,...,...,...,...,...,...,...,...
95,11,Île-de-France,95,Val-d'Oise,3,21.0,184,1228618,1239262
96,1,Guadeloupe,971,Guadeloupe,2,21.0,32,390253,396153
97,2,Martinique,972,Martinique,4,NaN,34,372594,377711
98,3,Guyane,973,Guyane,2,NaN,22,268700,271124


In [25]:
inseeDep.iloc[:,4:].sum()

Nombre d'arrondissements         332.0
Nombre de cantons               2041.0
Nombre de communes             34953.0
Population municipale       66524339.0
Population totale           67761092.0
dtype: float64

In [24]:
inseeReg.iloc[:,4:].sum()

Nombre de communes          34953
Population municipale    66524339
Population totale        67761092
dtype: int64

## Merging demographic data with Covid hospital or testing statistics
This should give a different perspective on those numbers.